# hw3
## Teammate: Huidong Xu, Boliang Liu

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm, bernoulli
import random
import statistics

# 1

In [ ]:
data = pd.read_csv('retail.csv')

In [ ]:
data.head()

,A,B
0,50.63,53.28
1,54.15,52.28
2,51.09,44.93
3,52.34,46.06
4,47.46,48.59


## (a)

In [ ]:
A_customers = data.A.values

In [ ]:
reject_times = 0

for _ in range(1000):
    # randomly split customers in “condition A” into A1 and A2
    A1 = []
    A2 = []
    for value in A_customers:
        group_index = random.choice([1, 2])
        if group_index == 1:
            A1.append(value)
        else:
            A2.append(value)

    # t test for mean difference mu_A1 and mu_A2
    n1 = len(A1)
    n2 = len(A2)
    mean1 = np.mean(A1)
    mean2 = np.mean(A2)
    s1_squared = statistics.variance(A1)
    s2_squared = statistics.variance(A2)
    pooled_variance = ((n1-1)*s1_squared + (n2-1)*s2_squared) / (n1+n2-2)
    t_stat = (mean1-mean2) / (pooled_variance * np.sqrt(1/n1 + 2/n2))
    pval = stats.t.sf(abs(t_stat), n1+n2-2)*2  # two sided

    if pval < 0.05: # reject H0
        reject_times += 1

In [ ]:
print('the number of times we reject the null hypothesis:', reject_times)

the number of times we reject the null hypothesis: 0


In [ ]:
proportion = reject_times / 1000
if proportion < 0.05: 
    print("Users assigned to A behave similarly.")    
else: 
    print("Reject H0. Users assigned to A don't behave similarly.")

Users assigned to A behave similarly.


## (b)

In [ ]:
A_prices = data.A.values
B_prices = data.B.values

H0: mu_A <= mu_B  
Ha: mu_A > mu_B    

In [ ]:
# t test 
n1 = len(A_prices)
n2 = len(B_prices)

mean1 = np.mean(A_prices)
mean2 = np.mean(B_prices)

s1_squared = statistics.variance(A_prices)
s2_squared = statistics.variance(B_prices)
pooled_variance = ((n1-1)*s1_squared + (n2-1)*s2_squared) / (n1+n2-2)

t_stat = (mean1-mean2) / (pooled_variance * np.sqrt(1/n1 + 2/n2))

pval = stats.t.sf(abs(t_stat), n1+n2-2)  # one sided

In [ ]:
if pval < 0.05: 
    print("Reject H0. We conclude that condition A results, on average, in significantly larger purchase prices than condition B, at a 5% level of significance. ")    
else: 
    print("Do not reject H0. Condition A results doesn't results significantly larger purchase prices than condition B. ")

Reject H0. We conclude that condition A results, on average, in significantly larger purchase prices than condition B, at a 5% level of significance. 


## (c)

In [ ]:
def sample_size_calculator_mean_test(α, δ, k, σ_2, power, alternative):
    """ α: alpha
        δ: effect size 
        k: sample-size multiplier
        σ2: the population variance
        power: desired power = 1-β
    """
    β = 1 - power
    z_β = stats.norm.ppf(1-(1-β))

    if alternative == 'not.equal':
        n2 = (1/k + 1) * (stats.norm.ppf(1-α/2) - z_β) ** 2 * σ_2 / (δ**2)
    else:
        n2 = (1/k + 1) * (stats.norm.ppf(1-α) - z_β) ** 2 * σ_2 / (δ**2)
        
    n1 = k * n2
    
    return int(n1), int(n2)

In [ ]:
α = 0.05
δ = 1
k = 1
σ_2 = 25
power = 0.9
alternative = 'greater.than'
sample_size_calculator_mean_test(α, δ, k, σ_2, power, alternative)

(428, 428)

# 2

In [ ]:
candy = pd.read_csv('candycrush.csv')
candy.head()

,time,booster
0,9.675618,3
1,4.446914,1
2,10.403742,3
3,6.693658,2
4,5.777017,2


## (a)

In [ ]:
booster_1 = candy[candy['booster'] == 1].time
booster_2 = candy[candy['booster'] == 2].time
booster_3 = candy[candy['booster'] == 3].time

+ test 1:  
H0: mu_1 >= mu_2  
Ha: mu_1 < mu_2   # booster 1 is less than booster 2
 
+ test 2:  
H0: mu_1 >= mu_3  
Ha: mu_1 < mu_3   # booster 1 is less then booster 3
  
+ test 3:  
H0: mu_2 >= mu_3  
Ha: mu_2 < mu_3   # booster 2 is less then booster 3

In [ ]:
def pairwise_t_test(s1, s2, adjusted_alpha):
    # t test 
    n1 = len(s1)
    n2 = len(s2)

    mean1 = np.mean(s1)
    mean2 = np.mean(s2)

    s1_squared = statistics.variance(s1)
    s2_squared = statistics.variance(s2)
    pooled_variance = ((n1-1)*s1_squared + (n2-1)*s2_squared) / (n1+n2-2)

    t_stat = (mean1-mean2) / (pooled_variance * np.sqrt(1/n1 + 2/n2))

    pval = stats.t.sf(abs(t_stat), n1+n2-2)  # one sided
    
    if pval < adjusted_alpha: 
        print("Reject H0.")    
    else: 
        print("Do not reject H0. ")
    return pval

In [ ]:
alpha = 0.05
adjusted_alpha = alpha / 3
adjusted_alpha

0.016666666666666666

### test 1:  
H0: mu_1 >= mu_2  
Ha: mu_1 < mu_2   # booster 1 is less than booster 2

In [ ]:
pval_1 = pairwise_t_test(booster_1, booster_2, adjusted_alpha)
print('Adjusted p-value for test 1 is', pval_1)

Reject H0.
Adjusted p-value for test 1 is 4.517698195380547e-22


There is evidence suggests mu_1 < mu_2.

### test 2:  
H0: mu_1 >= mu_3  
Ha: mu_1 < mu_3   # booster 1 is less than booster 3

In [ ]:
pval_2 = pairwise_t_test(booster_1, booster_3, adjusted_alpha)
print('Adjusted p-value for test 1 is', pval_2)

Reject H0.
Adjusted p-value for test 1 is 1.8016400121459934e-84


There is evidence suggests mu_1 < mu_3.



### test 3:  
H0: mu_2 >= mu_3  
Ha: mu_2 < mu_3   # booster 2 is less then booster 3

In [ ]:
pval_3 = pairwise_t_test(booster_2, booster_3, adjusted_alpha)
print('Adjusted p-value for test 1 is', pval_3)

Reject H0.
Adjusted p-value for test 1 is 2.3988524151994085e-66


There is evidence suggests mu_2 < mu_3.

So we get mu_1 < mu_2 < mu_3.  
Booster 3 is associated with the largest average length of game play.

In [ ]:
np.mean(booster_1), np.mean(booster_2), np.mean(booster_3)

(5.012806007768593, 6.188083192762961, 9.895591194393168)

## (b)

We have 3 booster conditions from which we'd like to optimize a metric -- playing time. First, we get an initial guess for the playing time by running at least one sample to each condition. Second, we choose greedy algorithm. Then assign next unit to the best condition and calculate the mean values for each condition until iterating all units. Finally, we can get the booster with the largest average length of game play

# 3

In [6]:
nike = pd.read_csv('nike.csv')

In [7]:
nike

,Region,Ad,Subject,Y
0,NE,1,1,0
1,NE,1,2,1
2,NE,1,3,1
3,NE,1,4,0
4,NE,1,5,0
...,...,...,...,...
99995,W,5,4996,1
99996,W,5,4997,1
99997,W,5,4998,0
99998,W,5,4999,0


In [8]:
for r in np.unique(nike['Region']):
    print(f'region {r}')
    for a in range(1,6):
        data = nike[(nike['Ad']==a) & (nike['Region']==r)]
        mean = np.mean(data['Y'])
        print("\tCTR of Ad {a} is {mean}".format(a=a, r=r, mean=mean))

region MW
	CTR of Ad 1 is 0.2084
	CTR of Ad 2 is 0.3172
	CTR of Ad 3 is 0.0976
	CTR of Ad 4 is 0.1984
	CTR of Ad 5 is 0.1978
region NE
	CTR of Ad 1 is 0.2964
	CTR of Ad 2 is 0.2024
	CTR of Ad 3 is 0.1028
	CTR of Ad 4 is 0.2938
	CTR of Ad 5 is 0.2044
region S
	CTR of Ad 1 is 0.1478
	CTR of Ad 2 is 0.1984
	CTR of Ad 3 is 0.1044
	CTR of Ad 4 is 0.206
	CTR of Ad 5 is 0.302
region W
	CTR of Ad 1 is 0.297
	CTR of Ad 2 is 0.209
	CTR of Ad 3 is 0.1052
	CTR of Ad 4 is 0.3016
	CTR of Ad 5 is 0.1988


In [9]:
def pairwise_z_test(s1, s2, alpha):
    # z test 
    n1 = len(s1)
    n2 = len(s2)

    pi_1_hat = len(s1[s1['Y']==1])/n1
    pi_2_hat = len(s2[s2['Y']==1])/n2

    pi_hat = (n1*pi_1_hat + n2*pi_2_hat) / (n1 + n2)

    t_stat = (pi_1_hat-pi_2_hat) / ((pi_hat*(1-pi_hat)*(1/n1+1/n2))**0.5)

    pval = stats.norm.sf(abs(t_stat))  # one sided
    
    if pval < alpha: 
        print("Reject H0.")    
    else: 
        print("Do not reject H0. ")
    return pval

## Region MW

In [10]:
nike_mw = nike[nike['Region']=='MW']

### test 1:  
H0: proportion_1 >= proportion_2  
Ha: proportion_1 < proportion_2 

In [11]:
pairwise_z_test(nike_mw[nike_mw['Ad']==1], nike_mw[nike_mw['Ad']==2], 0.05)

Reject H0.


2.1698381079114035e-35

### test 2:  
H0: proportion_4 >= proportion_1  
Ha: proportion_4 < proportion_1

In [12]:
pairwise_z_test(nike_mw[nike_mw['Ad']==4], nike_mw[nike_mw['Ad']==1], 0.05)

Do not reject H0. 


0.10709043488813086

### test 3:  
H0: proportion_5 >= proportion_1  
Ha: proportion_5 < proportion_1

In [13]:
pairwise_z_test(nike_mw[nike_mw['Ad']==5], nike_mw[nike_mw['Ad']==1], 0.05)

Do not reject H0. 


0.09385161192441599

### test 4:  
H0: proportion_1 <= proportion_3  
Ha: proportion_1 > proportion_3

In [17]:
pairwise_z_test(nike_mw[nike_mw['Ad']==1], nike_mw[nike_mw['Ad']==3], 0.05)

Reject H0.


9.637129489611536e-54

The highest CTR is Ad2 and the lowest CTR is Ad3

# 4

## (a)

In [ ]:
# ε-greedy
def epsilon_greedy(epsilon, conditions, units):
    k = len(conditions)
    n = units
    dic = {i:[] for i in range(k)}
    for i in range(k):
        dic[i].append(np.random.choice(conditions[i],1)[0])
    for j in range(k, n):
        p = np.random.binomial(1,1-epsilon)
        means = [np.mean(v) for s,v in dic.items()]
        means_max_index = means.index(np.max(means))
        if p == 1:
            new_assign = np.random.choice(conditions[means_max_index],1)[0]
            dic[means_max_index].append(new_assign)
        else:
            others = [i for i in range(k)]
            others.remove(means_max_index)
            choice = np.random.choice(others, 1)[0]
            new_assign = np.random.choice(conditions[choice],1)[0]
            dic[means_max_index].append(new_assign)
    return [round(np.mean(v),2) for u,v in dic.items()]
         

## (b)

In [19]:
# UCB
def ucb(conditions, units):
    k = len(conditions)
    n = units
    dic = {i:[] for i in range(k)}
    for i in range(k*10):
        i = i % k
        dic[i].append(np.random.choice(conditions[i],1)[0])
    for j in range(k*10, n):
        means = [np.mean(v) for s,v in dic.items()]
        stds = [np.std(v) for s,v in dic.items()]
        ucb = [means[i]+3*stds[i] for i in range(k)]
        arm = ucb.index(np.max(ucb))
        new_assign = np.random.choice(conditions[arm],1)[0]
        dic[arm].append(new_assign)   
    return [round(np.mean(v),2) for u,v in dic.items()]

## (c)

In [ ]:
# softmax
def softmax(conditions, units):
    k = len(conditions)
    n = units
    dic = {i:[] for i in range(k)}
    for i in range(k):
        dic[i].append(np.random.choice(conditions[i],1)[0])
    for j in range(k, n):
        means = [np.mean(v) for s,v in dic.items()]
        means_exp = [np.exp(m) for m in means]
        prob = np.array([np.exp(means[i])/sum(means_exp) for i in range(k)])
        arm = np.random.choice([i for i in range(k)], p = prob.ravel())
        new_assign = np.random.choice(conditions[arm],1)[0]
        dic[arm].append(new_assign)   
    return [round(np.mean(v),2) for u,v in dic.items()]

## Simulation

In [20]:
units = 1000
condition_1 = np.random.binomial(1,0.2,units)
condition_2 = np.random.binomial(1,0.5,units)
condition_3 = np.random.binomial(1,0.8,units)
conditions = [condition_1, condition_2, condition_3]

In [ ]:
epsilon_greedy(0.5, conditions, units)

[0.0, 0.5, 0.58]

Means of 3 arms are 0, 0.5, 0.58 respectively.

In [21]:
ucb(conditions, units)

[0.0, 0.31, 0.81]

Means of 3 arms are 0.1, 0.31, 0.81 respectively.

In [ ]:
softmax(conditions, units)

[0.2, 0.52, 0.79]

Means of 3 arms are 0.2, 0.52, 0.79 respectively.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=38d4ab31-3429-4a17-9b90-72da57a64579' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>